<a href="https://colab.research.google.com/github/q759729997/nlp_classification/blob/master/TNEWS_fastText.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

fastText is a library for efficient learning of word representations and sentence classification.

安装软件包

In [7]:
! pip install fasttext

In [0]:
# fasttext训练
# 参考：https://github.com/facebookresearch/fastText/tree/master/python
# 参考：https://pypi.org/project/fasttext/
# label prefix ['__label__']，标签与文本使用\t分割

读取数据

In [0]:
import pandas as pd
import random

In [9]:
# 连接云盘
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
df = pd.read_csv('/content/drive/My Drive/data/chineseGLUE/toutiao_category.csv')

In [11]:
df.head()

,data_type,news_id,category_code,category,title,keywords
0,dev,6552414358800957966,116,news_game,DNF：日常搬砖第32天，头条原创申请再次失败，卢克材料满100,"搬砖,格蓝迪,dnf,游戏,卢克"
1,dev,6553534223167258884,103,news_sports,如果詹姆斯最巅峰的时候出现了，杜兰特是否能够顶得住压力呢？,NaN
2,dev,6554376403674989070,104,news_finance,「大赛」节前埋伏高送转 坚定持有守得反包涨停！,"选手本周,上证综指,周收益"
3,dev,6553464292841095688,102,news_entertainment,盘点开演唱会被罚的歌手，五月天罚170万，被罚次数最多的是他,"周杰伦,陈奕迅,梁静茹,七里香,五月天,张学友"
4,dev,6554467498236838147,109,news_tech,联想把5G关键票投给了高通，掀起全国的舆论风暴，企业有权利按照自己意愿决择吗？,NaN


In [12]:
# 空数据处理
df = df.fillna('')
df.head()

,data_type,news_id,category_code,category,title,keywords
0,dev,6552414358800957966,116,news_game,DNF：日常搬砖第32天，头条原创申请再次失败，卢克材料满100,"搬砖,格蓝迪,dnf,游戏,卢克"
1,dev,6553534223167258884,103,news_sports,如果詹姆斯最巅峰的时候出现了，杜兰特是否能够顶得住压力呢？,
2,dev,6554376403674989070,104,news_finance,「大赛」节前埋伏高送转 坚定持有守得反包涨停！,"选手本周,上证综指,周收益"
3,dev,6553464292841095688,102,news_entertainment,盘点开演唱会被罚的歌手，五月天罚170万，被罚次数最多的是他,"周杰伦,陈奕迅,梁静茹,七里香,五月天,张学友"
4,dev,6554467498236838147,109,news_tech,联想把5G关键票投给了高通，掀起全国的舆论风暴，企业有权利按照自己意愿决择吗？,


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 382688 entries, 0 to 382687
Data columns (total 6 columns):
data_type        382688 non-null object
news_id          382688 non-null int64
category_code    382688 non-null int64
category         382688 non-null object
title            382688 non-null object
keywords         382688 non-null object
dtypes: int64(2), object(4)
memory usage: 17.5+ MB


In [27]:
# fasttext数据组装
train_data = list()
test_data = list()
line_count = len(df)
for i in range(line_count):
  if i % 10000 == 0:
    print('当前：{}，合计：{}'.format(i, line_count), end='\r')
  data_type = df.iloc[i]['data_type']
  category = df.iloc[i]['category']
  title = df.iloc[i]['title']
  keywords = df.iloc[i]['keywords']
  text = '{}{}'.format(title, keywords).replace('\t', '').strip()
  if len(text) < 2 or len(category) < 1:
    continue
  label = '__label__{}'.format(category)  # label prefix ['__label__']
  if data_type == 'test':
    test_data.append((label, text))
  else:
    train_data.append((label, text))

In [28]:
print(len(train_data))
print(len(test_data))

325284
57404


In [29]:
print(train_data[:2])
print(test_data[:2])

[('__label__news_game', 'DNF：日常搬砖第32天，头条原创申请再次失败，卢克材料满100搬砖,格蓝迪,dnf,游戏,卢克'), ('__label__news_sports', '如果詹姆斯最巅峰的时候出现了，杜兰特是否能够顶得住压力呢？')]
[('__label__news_military', '歼20座舱盖上的两条“花纹”是什么？'), ('__label__news_edu', '2018年2月退休教师，教龄38年，退休金是多少？')]


In [0]:
import codecs
# 数据写入文件
train_file = '/content/sample_data/toutiao_train_ft.txt'
with codecs.open(train_file, mode='w', encoding='utf8') as fw:
  for label, text in train_data:
    # line = '{}\t{}'.format(label, text)
    line = '{}\t{}'.format(label, ' '.join(list(text)))  # 使用单个字作为输入
    fw.write('{}\n'.format(line))
test_file = '/content/sample_data/toutiao_test_ft.txt'
with codecs.open(test_file, mode='w', encoding='utf8') as fw:
  for label, text in test_data:
    # line = '{}\t{}'.format(label, text)
    line = '{}\t{}'.format(label, ' '.join(list(text)))  # 使用单个字作为输入
    fw.write('{}\n'.format(line))

In [43]:
! head -5 /content/sample_data/toutiao_test_ft.txt

__label__news_military	歼 2 0 座 舱 盖 上 的 两 条 “ 花 纹 ” 是 什 么 ？
__label__news_edu	2 0 1 8 年 2 月 退 休 教 师 ， 教 龄 3 8 年 ， 退 休 金 是 多 少 ？
__label__news_military	第 8 1 集 团 军 某 旅 首 长 机 关 军 事 训 练 考 核 见 闻 军 事 训 练 , 2 0 1 8 年 度 军 事 训 练 工 作 指 示 , 首 长 , 风 雨 交 加 , 机 关 干 部 , 集 团 军
__label__news_military	王 阳 明 思 想 为 什 么 没 能 挽 救 大 明 王 朝 ？
__label__news_travel	一 生 中 一 定 要 去 的 八 个 地 方 ， 趁 着 年 轻 赶 紧 出 发 吧 茶 卡 盐 湖 , 泸 沽 湖 , 布 拉 格 , 圣 托 里 尼 , 圣 托 里 尼 岛


In [0]:
import fasttext

In [0]:
def print_results(N, p, r):
  print("N\t" + str(N))
  print("P@{}\t{:.3f}".format(1, p))
  print("R@{}\t{:.3f}".format(1, r))

In [49]:
# model = fasttext.train_supervised(input=train_file)
# model = fasttext.train_supervised(input=train_file, wordNgrams=2)
model = fasttext.train_supervised(input=train_file, wordNgrams=3)
print_results(*model.test(test_file))

N	57404
P@1	0.905
R@1	0.905


- 纯文本，未加分隔符，未分词
~~~
N	57404
P@1	0.348
R@1	0.348
~~~
- 单个字作为输入特征
~~~
N	57404
P@1	0.859
R@1	0.859
~~~
- 单个字作为输入特征,wordNgrams=2
~~~
N	57404
P@1	0.903
R@1	0.903
~~~

In [45]:
# 预测测试
model.predict("一起看周杰伦的演唱会")

(('__label__news_travel',), array([0.99733239]))

In [48]:
# 预测测试
model.predict(' '.join(list("一起看周杰伦的演唱会")))

(('__label__news_entertainment',), array([1.00000453]))

In [0]:
# 模型保存
model.save_model("/content/sample_data/toutiao_ft.ftz")